# Reprogramming Recepies

Auth: [Joshua Pickard](jpic@umich.edu)

Date: August 26, 2024

# Day 1

In [2]:
import numpy as np
import pandas as pd
import scanpy as sp
import os

## Perturbation Model Discussion

E.V. = expression values

Possible algorithm:
```
1. find highest E.V.  for a single cell
2. find expression value of TFs being modified
3. have a value k for the number of different concentrations we want to test
4. choose k different amounts to increase the TFs from there measured E.V. to the 150% maximum E.V.
   - make an arbitray choice and code it up
```

**A reasonable person could write this 10s of different ways**

In [3]:
import numpy as np
# median nonzero value of each row
# the w stands for working. I just dont want to screw up the original.
DATAPATH = "/nfs/turbo/umms-indikar/shared/projects/DGC/data/tabula_sapiens/extract/"
FILE = "TS_epithelial.h5ad"
adata_w = sp.read_h5ad(os.path.join(DATAPATH, FILE))
adata_w
# def median_nonzero(col):
#     nonzero_vals = col[col != 0]  # Extract nonzero values
#     if len(nonzero_vals) == 0:    # If no nonzero values, return NaN
#         return np.nan
#     return np.median(nonzero_vals)

# # Apply the function to each column and store the results
# med_nonz = np.apply_along_axis(median_nonzero, axis=0, arr=adata_w.X)
# adata_w.var['med_nonz'] = med_nonz
# adata_w.var

AnnData object with n_obs × n_vars = 104148 × 58870
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: '_scvi', '_training_mode', 'dendrogram_cell_type_tissue', 'dendrogram_computational_compartment_assignment', 'dendrogram_consensus_prediction', 'dendrogram_tissue_cell_type', 'donor_colors', 'donor_method_colors', 'hvg', 'method_colors', 'neighbors', 'organ_tissue_colors', 'sex_colors', 'tissue_colors', 'umap'
    obsm: 'X_pca', 'X_scvi', 'X_scvi_umap', 'X_umap'
    layers: 'decontXcounts', 'raw_counts'
    obsp: 'connectivities', 'distances'

In [4]:
import numpy as np


X = adata_w.X.toarray() if not isinstance(adata_w.X, np.ndarray) else adata_w.X

def median_nonzero(col):
    nonzero_vals = col[col != 0] 
    return np.median(nonzero_vals) if len(nonzero_vals) > 0 else 0

#perform and save results of fn
med_nonz = np.apply_along_axis(median_nonzero, axis=0, arr=X)
adata_w.var['med_nonz'] = med_nonz


In [5]:
adata_w.var.head()

,gene_symbol,feature_type,ensemblid,highly_variable,means,dispersions,dispersions_norm,mean,std,med_nonz
DDX11L1,DDX11L1,Gene Expression,ENSG00000223972.5,False,6.398244e-05,0.835044,-0.573947,0.000039,0.005574,6.029026
WASH7P,WASH7P,Gene Expression,ENSG00000227232.5,False,2.274395e-03,2.442280,0.533203,0.001080,0.031731,0.770418
MIR6859-1,MIR6859-1,Gene Expression,ENSG00000278267.1,False,6.175251e-05,1.295335,-0.256874,0.000033,0.005634,10.000000
MIR1302-2HG,MIR1302-2HG,Gene Expression,ENSG00000243485.5,False,1.372886e-04,2.656352,0.680668,0.000048,0.008041,5.042750
MIR1302-2,MIR1302-2,Gene Expression,ENSG00000284332.1,False,1.000000e-12,NaN,0.000000,0.000000,1.000000,0.000000


In [9]:
# seeing what it looks like before tf_list changes the first 3 rows
# Convert to dense if it's sparse and display the first five rows
import numpy as np

# Convert to a dense array if necessary
dense_X = adata_w.X.toarray() if not isinstance(adata_w.X, np.ndarray) else adata_w.X



[[0.        0.        0.        ... 4.490315  0.        0.       ]
 [0.        0.        0.        ... 4.4802847 0.        0.       ]
 [0.        0.        0.        ... 4.457717  0.        0.       ]
 ...
 [0.        0.        0.        ... 4.9418464 0.        0.       ]
 [0.        0.        0.        ... 4.8312063 0.        0.       ]
 [0.        0.        0.        ... 5.0140185 0.        0.       ]]


In [25]:
# problem, scaling by a factor of the max expressed gene in that cell means that you could be scaling by different genes for each cell,
# when the cells are all of the same type. for each get the median nonzero expression

# for testing purposes: v
tf_list = ['DDX11L1', 'WASH7P', 'MIR6859-1']
tf = 'DDX11L1'
# for testing purposes: ^

#mask = adata_w.obs_names.isin(tf_list)
mask = np.where(adata_w.var['gene_symbol'] == tf)[0]
adata_w.X[mask, :] = adata_w.X[mask, :] * adata_w.var['med_nonz'].values 



/sw/pkgs/arc/python3.11-anaconda/2024.02-1/lib/python3.11/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [26]:
DATAPATH = "/nfs/turbo/umms-indikar/shared/projects/DGC/data/tabula_sapiens/extract/"
FILE = "TS_epithelial.h5ad"
adata = sp.read_h5ad(os.path.join(DATAPATH, FILE))
(adata_w.X[mask, :] - adata.X[mask, :]).sum()

2140043500.0

In [21]:
adata_w.X[mask, :]

<0x58870 sparse matrix of type '<class 'numpy.float32'>'
	with 0 stored elements in Compressed Sparse Row format>

In [20]:
mask


array([False, False, False, ..., False, False, False])

In [ ]:
############

In [ ]:

import numpy as np
DATAPATH = "/nfs/turbo/umms-indikar/shared/projects/DGC/data/tabula_sapiens/extract/"
FILE = "TS_epithelial.h5ad"
adata_w = sp.read_h5ad(os.path.join(DATAPATH, FILE))
#tf = 'DDX11L1'
tf_list =['DDX11L1', 'WASH7P', 'MIR6859-1']

def median_nonzero(col):
    nonzero_vals = col[col != 0] 
    return np.median(nonzero_vals) if len(nonzero_vals) > 0 else 0

# requires scalar is a scalar
def perturb_counts(tf_list, scalar, adata): 
    # compute nonzero median expression of each gene across cells, save to var
    med_nonz = np.apply_along_axis(median_nonzero, axis=0, arr=adata.X)
    adata.var['med_nonz'] = med_nonz

    # filter by desired tf(s), and apply the nonzero_median scaling operation to only these  
    mask = np.where(adata.var['gene_symbol'].isin(tf_list))[0]
    adata.X[mask, :] = adata.X[mask, :] * adata.var['med_nonz'].values * scalar
    return adata


#### 1. 

In [ ]:
############

## Visualize Input Data

In [2]:
DATAPATH = "/nfs/turbo/umms-indikar/shared/projects/DGC/data/tabula_sapiens/extract/"
FILE = "TS_epithelial.h5ad"

adata = sp.read_h5ad(os.path.join(DATAPATH, FILE))

In [3]:
adata

AnnData object with n_obs × n_vars = 104148 × 58870
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: '_scvi', '_training_mode', 'dendrogram_cell_type_tissue', 'dendrogram_computational_compartment_assignment', 'dendrogram_consensus_prediction', 'dendrogram_tissue_cell_type', 'donor_colors', 'donor_method_colors', 'hvg', 'method_colors', 'neighbors', 'organ_tissue_colors', 'sex_colors', 'tissue_colors', 'umap'
    obsm: 'X_pca', 'X_scvi', 'X_scvi_umap', 'X_umap'
    layers: 'decontXcounts', 'raw_counts'
    obsp: 'connectivities', 'distances'

In [4]:
adata.var

,gene_symbol,feature_type,ensemblid,highly_variable,means,dispersions,dispersions_norm,mean,std
DDX11L1,DDX11L1,Gene Expression,ENSG00000223972.5,False,6.398244e-05,0.835044,-0.573947,0.000039,0.005574
WASH7P,WASH7P,Gene Expression,ENSG00000227232.5,False,2.274395e-03,2.442280,0.533203,0.001080,0.031731
MIR6859-1,MIR6859-1,Gene Expression,ENSG00000278267.1,False,6.175251e-05,1.295335,-0.256874,0.000033,0.005634
MIR1302-2HG,MIR1302-2HG,Gene Expression,ENSG00000243485.5,False,1.372886e-04,2.656352,0.680668,0.000048,0.008041
MIR1302-2,MIR1302-2,Gene Expression,ENSG00000284332.1,False,1.000000e-12,NaN,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...
MT-ND6,MT-ND6,Gene Expression,ENSG00000198695.2,False,9.634841e-01,2.466404,0.154140,0.590065,0.741395
MT-TE,MT-TE,Gene Expression,ENSG00000210194.1,False,1.600667e-01,1.603787,-0.044396,0.083929,0.301820
MT-CYB,MT-CYB,Gene Expression,ENSG00000198727.2,False,4.367693e+00,4.765751,-0.499747,3.874830,1.104192
MT-TT,MT-TT,Gene Expression,ENSG00000210195.2,False,6.573967e-02,0.624316,-0.719108,0.040580,0.186848


In [5]:
adata.X.max(axis=1) # what is the value of the highest expressed gene for each cell?

<104148x1 sparse matrix of type '<class 'numpy.float32'>'
	with 104148 stored elements in COOrdinate format>

In [6]:
TF = 'DDX11L1'
index = np.where(adata.var['gene_symbol'] == TF)[0]
index

array([0])

In [7]:
adata.var['gene_symbol']

DDX11L1            DDX11L1
WASH7P              WASH7P
MIR6859-1        MIR6859-1
MIR1302-2HG    MIR1302-2HG
MIR1302-2        MIR1302-2
                  ...     
MT-ND6              MT-ND6
MT-TE                MT-TE
MT-CYB              MT-CYB
MT-TT                MT-TT
MT-TP                MT-TP
Name: gene_symbol, Length: 58870, dtype: category
Categories (57316, object): ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', ..., 'ZYX', 'ZYXP1', 'ZZEF1', 'hsa-mir-1253']

In [8]:
adata.obs

,organ_tissue,method,donor,anatomical_information,n_counts_UMIs,n_genes,cell_ontology_class,free_annotation,manually_annotated,compartment,gender
AACAGGGCATGTGCTA_TSP6_Liver_NA_10X_1_1,Liver,10X,TSP6,nan,14151.0,3404,hepatocyte,Hepatocyte,True,epithelial,male
AATGAAGTCTAAGGAA_TSP6_Liver_NA_10X_1_1,Liver,10X,TSP6,nan,10303.0,2863,hepatocyte,Hepatocyte,True,epithelial,male
ACAGAAAAGCTGTCCG_TSP6_Liver_NA_10X_1_1,Liver,10X,TSP6,nan,9391.0,2738,hepatocyte,Hepatocyte,True,epithelial,male
ACGATGTTCGACTCCT_TSP6_Liver_NA_10X_1_1,Liver,10X,TSP6,nan,4634.0,1634,hepatocyte,Hepatocyte,True,epithelial,male
ACGTTCCAGAACCCGA_TSP6_Liver_NA_10X_1_1,Liver,10X,TSP6,nan,33370.0,6590,intrahepatic cholangiocyte,BECS,True,epithelial,male
...,...,...,...,...,...,...,...,...,...,...,...
TGACAGTAGATGGCGT_TSP2_Vasculature_Aorta_10X_1_2,Vasculature,10X,TSP2,Aorta,10354.0,2456,epithelial cell,epithelial cell,True,epithelial,female
TGTTGGATCGACATCA_TSP2_Vasculature_Aorta_10X_1_2,Vasculature,10X,TSP2,Aorta,15677.0,3234,epithelial cell,epithelial cell,True,epithelial,female
TTGGATGGTGGCTACC_TSP2_Vasculature_Aorta_10X_1_2,Vasculature,10X,TSP2,Aorta,18638.0,3368,epithelial cell,epithelial cell,True,epithelial,female
CTTGATTTCTTGCAAG_TSP2_Vasculature_Aorta_10X_2_2,Vasculature,10X,TSP2,Aorta,36555.0,5102,epithelial cell,epithelial cell,True,epithelial,female


## Build driver

In [9]:
import pandas as pd

def main(job_number, parameter_file):
    """
    This is the main function for the array job to perform the reprogramming experiment. job_number is a single parameter
    that will be used to look up in a parameter table which model, reprogramming recipe, and other information relevant
    to the test.
    """

    # Determine embedding parameters and recipie
    df_embedding_parameters = pd.read_csv(parameter_file)
    TFs    = df_embedding_parameters['TFs'].values[job_number]
    model  = df_embedding_parameters['model'].values[job_number]
    source = df_embedding_parameters['source'].values[job_number]
    target = df_embedding_parameters['target'].values[job_number]

    # Load the source data
    adata = 

    # Perturb the data
    perturbed_adata = perturbation_model(adata, TFs)

    # Generate embeddings
    if model == 'geneformer':
        adata_embedded = embed_geneformer([source_adata, perturbed_adata, target_adata])
    elif model == 'tGPT':
        adata_embedded = embed_tGPT([source_adata, perturbed_adata, target_adata])
    elif model == 'scGTP':
        adata_embedded = embed_scGTP([source_adata, perturbed_adata, target_adata])

    # Save the results to a file

    return 0


SyntaxError: invalid syntax (1649016643.py, line 18)

## Build parameter dataframe

In [ ]:
embedding_parameters = {
    'source': [],
    'target': [],
    'TFs'   : [],
    'model' : []
}
models = ['geneformer', 'tGPT', 'scGTP']

df = pd.read_csv('data/first_5_recepies_8_29_2024.csv')

for i in range(5):
    TFs = df['TFs'].values[i].split()
    source = df['Source'].values[i]
    target = df['Target'].values[i]
    for model in models:
        embedding_parameters['TFs'].append(TFs)
        embedding_parameters['source'].append(source)
        embedding_parameters['target'].append(source)
        embedding_parameters['model'].append(model)

df_embedding_parameters = pd.DataFrame(embedding_parameters)

df_embedding_parameters

In [ ]:
df

# Day 0

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load known reprogramming regiems
df = pd.read_csv('data/known-regiems-T1.csv')

In [ ]:
# get list of unique transcription factors
TFs = []
for regime in df['TFs'].unique():
    TFs += regime.replace(',', '').split()
TFs = list(set(TFs))
print(f"{len(TFs)=}")
TFs